**ДЗ 2 к уроку 2. Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)**

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [51]:
import pandas as pd

news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Наши новости

In [52]:
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [53]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [54]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [55]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [57]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

151

In [58]:
morph = pymorphy2.MorphAnalyzer()

In [59]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [60]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [11]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


Wall time: 33 s


In [12]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 4min 39s


А теперь в 3 строчки обучим нашу модель

In [13]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [14]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [15]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min


In [16]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [17]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(2, 0.40610397),
 (3, 0.02281739),
 (15, 0.21605447),
 (21, 0.11441953),
 (22, 0.07724001),
 (23, 0.14654617)]

In [18]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: взрыв фотография дыра знаменитый пища питание мэй
topic_1: экипаж место nn км высота министерство двигатель
topic_2: космос разместить студент кость университет тур первый
topic_3: земля смерть пациент доллар операция лечение врач
topic_4: военный это риск исследование американский сша мочь
topic_5: год украина президент россия nn глава который
topic_6: выдавать миллиардер мэр кит популярный втб свести
topic_7: продукция энергия выдать киев соцсеть спровоцировать сотрудничать
topic_8: восток российский закон обращение законопроект флот озеро
topic_9: исследование фонд население журнал год дом nn
topic_10: развитие программа доля новый правительство налог система
topic_11: год газ который человек это район россия
topic_12: nn ребёнок человек работа который это научный
topic_13: доклад занятость эндрю рот володин украсить леонтьев
topic_14: станция кожа еда вход открыться золото метро
topic_15: год это который весь мочь новый свой
topic_16: рейс турция турецкий россия путин план

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [19]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [20]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.00000,0.088620,0.0,0.0,0.364346,0.000000,0.0,0.0,...,0.000000,0.014678,0.043070,0.000000,0.0,0.0,0.0,0.481974,0.000000,0.0
1,4896,0.282182,0.00000,0.441618,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.110627,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,4897,0.000000,0.00000,0.416954,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.215432,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.159924,0.154343,0.0
3,4898,0.000000,0.05652,0.416630,0.0,0.0,0.000000,0.011521,0.0,0.0,...,0.505184,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
4,4899,0.021897,0.00000,0.000000,0.0,0.0,0.584437,0.000000,0.0,0.0,...,0.000000,0.000000,0.040786,0.329503,0.0,0.0,0.0,0.000000,0.000000,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [21]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [22]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [23]:
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.03295816, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01807838, 0.17513447, 0.        , 0.01628049, 0.        ,
       0.        , 0.06397789, 0.08727501, 0.        , 0.28896815,
       0.        , 0.28386289, 0.02375025, 0.        , 0.        ])

In [24]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [25]:
get_user_embedding(user_articles_list)

array([0.        , 0.01003053, 0.00997821, 0.00234655, 0.06728963,
       0.15534715, 0.        , 0.        , 0.0379403 , 0.07470248,
       0.02826314, 0.07557297, 0.01746302, 0.        , 0.        ,
       0.07456005, 0.04966147, 0.25979531, 0.        , 0.05193871,
       0.01027862, 0.0486774 , 0.01408048, 0.        , 0.        ])

Интересовался новостями с топиками topic_10, topic_14, topic_16 

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [26]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.01033,0.000000,0.00000,0.005493,0.005012,0.183526,0.0,0.009685,0.000000,...,0.119976,0.044104,0.147415,0.000000,0.066305,0.006014,0.073393,0.012452,0.002780,0.000000
1,u108690,0.00000,0.016765,0.00000,0.006947,0.021316,0.126772,0.0,0.006238,0.002341,...,0.164930,0.031042,0.275896,0.000000,0.023940,0.000000,0.025416,0.025621,0.011462,0.015986
2,u108339,0.00000,0.009611,0.01242,0.006588,0.012168,0.066744,0.0,0.000000,0.006684,...,0.035266,0.000000,0.292648,0.007208,0.036876,0.000000,0.056019,0.038497,0.027791,0.002086


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [27]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [28]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.01033,0.000000,0.00000,0.005493,0.005012,0.183526,0.0,0.009685,0.000000,...,0.044104,0.147415,0.000000,0.066305,0.006014,0.073393,0.012452,0.002780,0.000000,0
1,u108690,0.00000,0.016765,0.00000,0.006947,0.021316,0.126772,0.0,0.006238,0.002341,...,0.031042,0.275896,0.000000,0.023940,0.000000,0.025416,0.025621,0.011462,0.015986,1
2,u108339,0.00000,0.009611,0.01242,0.006588,0.012168,0.066744,0.0,0.000000,0.006684,...,0.000000,0.292648,0.007208,0.036876,0.000000,0.056019,0.038497,0.027791,0.002086,1


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [30]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [31]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [32]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.36565123, 0.01204237, 0.45139956, 0.19864409, 0.00609074,
       0.04181899, 0.16661578, 0.01318128, 0.11713766, 0.13862282])

In [33]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [34]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
thresholds_mean = thresholds[ix]
fscore_mean = fscore[ix]
precision_mean = precision[ix]
recall_mean = recall[ix]
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.727611, F-Score=nan, Precision=0.000, Recall=0.000


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [35]:
roc_auc_score_mean = roc_auc_score(y_test, preds)
roc_auc_score_mean

0.9406128263271121

В целом мы видим, что получившиеся векторные представления содержат какой-то сигнал и позволяют решать нашу прикладную задачу. 


## Попробуем заменить параметр на медиану

In [36]:
def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [37]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.0,0.000000,0.000000,0.0,0.000000,0.072820,0.0,0.0,0.0,...,0.106472,0.000000,0.092025,0.0,0.000000,0.0,0.016650,0.007832,0.000000,0.0
1,u108690,0.0,0.000000,0.000000,0.0,0.006205,0.134973,0.0,0.0,0.0,...,0.145735,0.009226,0.297309,0.0,0.000000,0.0,0.000000,0.025881,0.000000,0.0
2,u108339,0.0,0.007101,0.006532,0.0,0.000000,0.030149,0.0,0.0,0.0,...,0.020603,0.000000,0.299991,0.0,0.037674,0.0,0.012476,0.000000,0.022989,0.0


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [38]:
target = pd.read_csv("users_churn.csv")
X = pd.merge(user_embeddings, target, 'left')

In [39]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [40]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [41]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.33397805, 0.00841769, 0.35113089, 0.15562815, 0.009009  ,
       0.02805071, 0.16938121, 0.00995275, 0.10254144, 0.09105892])

### Рассчитаем Precision, Recall, F_score

In [42]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
roc_auc_score_median = roc_auc_score(y_test, preds)
thresholds_median = thresholds[ix]
fscore_median = fscore[ix]
precision_median = precision[ix]
recall_median = recall[ix]
print(f'Threshold={round(thresholds_median,4)}, F-Score={round(fscore_median, 4)}, Precision={round(precision_median,4)}, Recall={round(recall_median,4)}, roc_auc_score={round(roc_auc_score_median,4)}')

Threshold=0.3623, F-Score=0.7404, Precision=0.7733, Recall=0.7102, roc_auc_score=0.9613


**С медианным параметром показатели получились лучше, в предыдущем были нули, здесь Precision=0.7733, Recall=0.7102. Также выше roc_auc_score=0.9613**

**Попробуем заменить параметр на максимальное значение**

In [43]:
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [44]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.061977,0.000000,0.000000,0.032958,0.030072,0.560003,0.0,0.058109,0.000000,...,0.302459,0.200649,0.389025,0.000000,0.288968,0.036084,0.283863,0.035298,0.016680,0.000000
1,u108690,0.000000,0.084216,0.000000,0.041681,0.071678,0.217781,0.0,0.019111,0.014044,...,0.463101,0.117900,0.453545,0.000000,0.127411,0.000000,0.112777,0.049758,0.068772,0.067003
2,u108339,0.000000,0.023775,0.039219,0.023796,0.073006,0.213907,0.0,0.000000,0.027514,...,0.122731,0.000000,0.429767,0.021989,0.084240,0.000000,0.252795,0.130111,0.073011,0.012516


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [45]:
target = pd.read_csv("users_churn.csv")
X = pd.merge(user_embeddings, target, 'left')

In [46]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [47]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [48]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.39070546, 0.00436722, 0.76891565, 0.26879377, 0.01591522,
       0.08873388, 0.07016544, 0.00394519, 0.01142572, 0.24719327])

### Рассчитаем Precision, Recall, F_score

In [49]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
roc_auc_score_max = roc_auc_score(y_test, preds)
thresholds_max = thresholds[ix]
fscore_max = fscore[ix]
precision_max = precision[ix]
recall_max = recall[ix]
print(f'Threshold={round(thresholds_max,4)}, F-Score={round(fscore_max, 4)}, Precision={round(precision_max,4)}, Recall={round(recall_max,4)}, roc_auc_score={round(roc_auc_score_max,4)}')

Threshold=0.4376, F-Score=0.7914, Precision=0.8364, Recall=0.751, roc_auc_score=0.9685


**В данном случае F-Score=0.7914, Precision=0.8364, Recall=0.751, roc_auc_score=0.9685**

In [50]:
result_table = pd.DataFrame({'fscore':[fscore_mean, fscore_median, fscore_max], 'Precision':[precision_mean, precision_median, precision_max], 'Recall':[recall_mean, recall_median, recall_max], 'roc_auc_score':[roc_auc_score_mean, roc_auc_score_median, roc_auc_score_max]}, index=['mean', 'median', 'max'])
result_table 

,fscore,Precision,Recall,roc_auc_score
mean,NaN,0.000000,0.000000,0.940613
median,0.740426,0.773333,0.710204,0.961326
max,0.791398,0.836364,0.751020,0.968463


**С каждым дальнейшим методом растет точность и полнота. Использование median и max оказались более удачными с точки зрения fscore и roc_auc_score. Median и max скорее всего больше отражают реальность. При использовании median - коэффициенты заинтересованности пользователя меньше, есть 0, которые попадают в медианные значения.**

**По представленной таблице выше, предлагается использовать max, где общая заинтересованность пользователя во всех темах будут выше, так как берутся максимальные значения.**